In [19]:
import requests
from pathlib import Path
import sys
import pandas as pd
src_path = Path.cwd().parent / 'src'
sys.path.append(str(src_path))
import mle_utils as mUtil
import carball
from carball.json_parser.game import Game
from carball.analysis.analysis_manager import AnalysisManager

In [2]:
data_path = Path.cwd().parent / 'data'
secrets_path = data_path / 'secrets.txt'
secrets_file = open(str(secrets_path), mode='r')
auth_token = secrets_file.readline()
secrets_file.close()
s11_csv_path = data_path / 's11_raw_replay_data.csv'

pulling in s11 raw replay spreadsheet produced by mle stats team using ballchasing.com api

In [3]:
s11_spreadsheet = mUtil.read_multiindex_csv_to_df(s11_csv_path)

retreive all categories from the data

In [7]:
categories = []
for item in list(s11_spreadsheet.columns):
    if not item[0] in categories:
        categories.append(item[0])
categories

['Series',
 'Series Game',
 'Series Replay',
 'Team',
 'Team Stats Core',
 'Team Stats Ball',
 'Player Data',
 'Player ID',
 'Player Camera',
 'Player Stats Core',
 'Player Stats Boost',
 'Player Stats Movement',
 'Player Stats Positioning',
 'Player Stats Demo']

time to try and recreate the first 4 lines of the dataframe using the raw replay and the carball library

In [14]:
replay_filename = s11_spreadsheet['Series Replay'].iloc[0]['id'] + '.replay'
replay_filename

'6612902d-039b-4ec2-91e6-6036dcb7c0aa.replay'

In [18]:
cb_json = carball.decompile_replay(str(data_path / 's11_replays' / replay_filename))

initialize carball.game object with the replay file and create an analysis manager using that game

In [20]:
game = Game()
game.initialize(loaded_json=cb_json)

analysis_manager = AnalysisManager(game)
analysis_manager.create_analysis()
analysis_df = analysis_manager.get_data_frame()
analysis_df

Replay name not found
Player Sleegi as player has no MatchScore.
Score is not found for player
Ignoring player: Sleegi as player has no team.
Player SimpleAOB as player has no MatchScore.
Score is not found for player
Ignoring player: SimpleAOB as player has no team.
Player FOX | Thunderballs as player has no MatchScore.
Score is not found for player
Ignoring player: FOX | Thunderballs as player has no team.
Player RJsaurus as player has no MatchScore.
Score is not found for player
Ignoring player: RJsaurus as player has no team.
Dropping these columns[('game', 'is_overtime')]
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


FLA | TyTy                                                     \
            ping        pos_x        pos_y       pos_z        vel_x   
1              7   256.000000  3840.000000   17.049999     0.000000   
2              7   256.000000  3840.000000   17.049999     0.000000   
3              7   256.000000  3840.000000   17.049999     0.000000   
4              7   256.000000  3840.000000   17.049999     0.000000   
5              7   256.000000  3840.000000   17.049999     0.000000   
...          ...          ...          ...         ...          ...   
10608          7 -2537.729980  -759.869995  182.580002 -5120.399780   
10609          7 -2574.120117  -928.640015  192.750000 -4593.699951   
10610          7 -2574.120117  -928.640015  192.750000 -4593.699951   
10611          7 -2602.699951 -1079.449951  199.240005 -4019.599915   
10612          7 -2602.699951 -1079.449951  199.240005 -4019.599915   

                                                                       ...  \
              vel_y        vel_z  ang_vel_x    ang_vel_y    ang_vel_z  ...   
1          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
2          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
3          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
4          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
5          0.000000     3.200000   0.700000     0.000000     0.000000  ...   
...             ...          ...        ...          ...          ...  ...   
10608 -22367.700195  1569.799957 -88.999996   615.200005  3183.599854  ...   
10609 -22505.700684  1177.699966 -86.999998  1267.399979  3448.299866  ...   
10610 -22505.700684  1177.699966 -86.999998  1267.399979  3448.299866  ...   
10611 -22631.599121   806.500015 -30.699999  2286.100006  3267.699890  ...   
10612 -22631.599121   806.500015 -30.699999  2286.100006  3267.699890  ...   

             game                                                           \
             time     delta seconds_remaining replicated_seconds_remaining   
1      182.439346  0.037653               300                          NaN   
2      182.472687  0.033334               300                          NaN   
3      182.506027  0.033334               300                          NaN   
4      182.539368  0.033334               300                          NaN   
5      182.572708  0.033334               300                          NaN   
...           ...       ...               ...                          ...   
10608  596.774475  0.033335                 0                          NaN   
10609  596.810852  0.036405                 0                          NaN   
10610  596.844360  0.033561                 0                          NaN   
10611  596.877625  0.033334                 0                          NaN   
10612  596.915100  0.037517                 0                          NaN   

                                       FLA | TyTy  PAN // antzu PAN // Unborn  \
      ball_has_been_hit goal_number boost_collect boost_collect boost_collect   
1                  None         NaN           NaN           NaN           NaN   
2                  None         NaN           NaN           NaN           NaN   
3                  None         NaN           NaN           NaN           NaN   
4                  None         NaN           NaN           NaN           NaN   
5                  None         NaN           NaN           NaN           NaN   
...                 ...         ...           ...           ...           ...   
10608              True        -1.0           NaN           NaN           NaN   
10609              True        -1.0           NaN           NaN           NaN   
10610              True        -1.0           NaN           NaN           NaN   
10611              True        -1.0           NaN           NaN           NaN   
10612              True        -1.0           NaN           NaN           NaN   